In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that have been scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://www.uhc.com/")
ed.links

['#main-content',
 '/shop-online-health-insurance-plans',
 '/insurance-plan-finder',
 'https://www.uhc.com/medicare',
 'https://www.uhc.com/communityplan',
 '/individuals-families',
 '/employer',
 '/individuals-families/aca-marketplace',
 '/individuals-families',
 'https://www.uhc.com/medicare',
 'https://www.uhc.com/communityplan',
 '/dental-vision-supplemental-plans',
 '/sign-in',
 '/member-resources/health-care-tools/unitedhealthcare-app',
 'https://unitedhealthcare.app.link/viXCFRARhFb',
 '/understanding-health-insurance/understanding-health-insurance-costs/flexible-spending-accounts',
 '/understanding-health-insurance/how-does-health-insurance-work',
 'https://www.uhc.com/medicare',
 'https://www.uhc.com/communityplan',
 '/individuals-families/aca-marketplace',
 '/individuals-families/health-plans-through-work',
 '/understanding-health-insurance/open-enrollment',
 '/individuals-families/aca-marketplace',
 'https://www.uhc.com/medicare',
 'https://www.uhc.com/communityplan',
 '/ind

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "plans page": "url": "https://another.full.url/plans"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "plans page": "url": "https://another.full.url/plans"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.uhc.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#main-content
/shop-online-health-insurance-plans
/insurance-plan-finder
https://www.uhc.com/medicare
https://www.uhc.com/communityplan
/individuals-families
/employer
/individuals-families/aca-marketplace
/individuals-families
https://www.uhc.com/medicare
https://www.uhc.com/communityplan
/dental-vision-supplemental-plans
/sign-in
/member-resources/health-care-tools/unitedhealthcare-app
https://unitedhealthcare.app.link/viXCFRARhFb
/understanding-health-insurance/understanding-health-insurance-costs/flexible-spending-accounts
/understanding-health-insurance/how-does-health-insurance-work
https://www.uhc.com/medicare
https://www.uhc.com/communityplan
/individuals-families/aca-marketplace
/individu

In [11]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [12]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://www.uhc.com/")
huggingface.links

['#main-content',
 '/shop-online-health-insurance-plans',
 '/insurance-plan-finder',
 'https://www.uhc.com/medicare',
 'https://www.uhc.com/communityplan',
 '/individuals-families',
 '/employer',
 '/individuals-families/aca-marketplace',
 '/individuals-families',
 'https://www.uhc.com/medicare',
 'https://www.uhc.com/communityplan',
 '/dental-vision-supplemental-plans',
 '/sign-in',
 '/member-resources/health-care-tools/unitedhealthcare-app',
 'https://unitedhealthcare.app.link/viXCFRARhFb',
 '/understanding-health-insurance/understanding-health-insurance-costs/flexible-spending-accounts',
 '/understanding-health-insurance/how-does-health-insurance-work',
 'https://www.uhc.com/medicare',
 'https://www.uhc.com/communityplan',
 '/individuals-families/aca-marketplace',
 '/individuals-families/health-plans-through-work',
 '/understanding-health-insurance/open-enrollment',
 '/individuals-families/aca-marketplace',
 'https://www.uhc.com/medicare',
 'https://www.uhc.com/communityplan',
 '/ind

In [13]:
get_links("https://www.uhc.com/")

{'links': [{'type': 'medicare page', 'url': 'https://www.uhc.com/medicare'},
  {'type': 'community plan page', 'url': 'https://www.uhc.com/communityplan'},
  {'type': 'individuals and families page',
   'url': 'https://www.uhc.com/individuals-families'},
  {'type': 'employer page', 'url': 'https://www.uhc.com/employer'},
  {'type': 'aca marketplace page',
   'url': 'https://www.uhc.com/individuals-families/aca-marketplace'},
  {'type': 'dental and vision plans page',
   'url': 'https://www.uhc.com/dental-vision-supplemental-plans'},
  {'type': 'understanding health insurance page',
   'url': 'https://www.uhc.com/understanding-health-insurance'},
  {'type': 'news articles page', 'url': 'https://www.uhc.com/news-articles'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://www.uhc.com/"))

Found links: {'links': [{'type': 'medicare page', 'url': 'https://www.uhc.com/medicare'}, {'type': 'community plan page', 'url': 'https://www.uhc.com/communityplan'}, {'type': 'individuals and families page', 'url': 'https://www.uhc.com/individuals-families'}, {'type': 'employer page', 'url': 'https://www.uhc.com/employer'}, {'type': 'aca marketplace page', 'url': 'https://www.uhc.com/individuals-families/aca-marketplace'}, {'type': 'dental vision supplemental plans page', 'url': 'https://www.uhc.com/dental-vision-supplemental-plans'}, {'type': 'understanding health insurance page', 'url': 'https://www.uhc.com/understanding-health-insurance'}, {'type': 'news articles page', 'url': 'https://www.uhc.com/news-articles'}]}
Landing page:
Webpage Title:
Health insurance plans | UnitedHealthcare
Webpage Contents:
Skip to main content
Health insurance plans
Health plans for life's moments
Our plans are made to deliver what matters, from quality coverage to caring support. Find a plan option th

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
print(get_brochure_user_prompt("United HealthCare", "https://www.uhc.com/"))

Found links: {'links': [{'type': 'medicare page', 'url': 'https://www.uhc.com/medicare'}, {'type': 'community plan page', 'url': 'https://www.uhc.com/communityplan'}, {'type': 'individuals and families page', 'url': 'https://www.uhc.com/individuals-families'}, {'type': 'employer page', 'url': 'https://www.uhc.com/employer'}, {'type': 'aca marketplace page', 'url': 'https://www.uhc.com/individuals-families/aca-marketplace'}, {'type': 'dental vision supplemental plans page', 'url': 'https://www.uhc.com/dental-vision-supplemental-plans'}, {'type': 'understanding health insurance page', 'url': 'https://www.uhc.com/understanding-health-insurance'}, {'type': 'news articles page', 'url': 'https://www.uhc.com/news-articles'}]}
You are looking at a company called: United HealthCare
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.
Landing page:
Webpage Title:
Health insurance plans | UnitedHealthcare
We

In [22]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [23]:
create_brochure("United HealthCare", "https://www.uhc.com/")

Found links: {'links': [{'type': 'medicare page', 'url': 'https://www.uhc.com/medicare'}, {'type': 'community plan page', 'url': 'https://www.uhc.com/communityplan'}, {'type': 'individuals and families page', 'url': 'https://www.uhc.com/individuals-families'}, {'type': 'employer page', 'url': 'https://www.uhc.com/employer'}, {'type': 'dental and vision plans page', 'url': 'https://www.uhc.com/dental-vision-supplemental-plans'}, {'type': 'understanding health insurance page', 'url': 'https://www.uhc.com/understanding-health-insurance'}, {'type': 'news articles page', 'url': 'https://www.uhc.com/news-articles'}]}


```markdown
# UnitedHealthcare: Health Insurance Plans

## Overview
At UnitedHealthcare, we empower individuals and families with comprehensive health insurance solutions tailored to life's moments. Our mission is to ensure that quality coverage and caring support are accessible to everyone.

## Health Plans for Every Stage of Life
Explore a variety of health plans designed to meet the unique needs of various demographics, including:

- **Medicare**: Options that go beyond Original Medicare, featuring Medicare Advantage plans for enhanced coverage.
- **Medicaid**: Affordable plans for those with low income or who qualify under special circumstances.
- **Individual & Family Plans**: ACA Marketplace plans available during the Open Enrollment Period (OEP) from November 1 to January 15.
- **Dental and Vision Plans**: Additional coverage to help with medical costs not included in your primary plan.

## Customer-Centric Approach
We believe in a personalized approach to healthcare. Our members can benefit from:
- **User-Friendly Access**: The UnitedHealthcare app provides instant access to plan details, member ID cards, care options, and more.
- **Financial Tools**: Learn about using your Flexible Spending Accounts (FSA) effectively to maximize your healthcare spending.

## Commitment to Wellbeing
Health insurance is not just for illnesses; it plays a vital role in preventive care and wellness. Our plans are designed to help members save money by addressing potential health concerns early.

## Company Culture
At UnitedHealthcare, our culture is rooted in compassion and support for our members. We are committed to building a diverse and inclusive workplace where every employee can thrive. Collaboration and innovation are at the forefront of our organizational values, creating an environment that nurtures personal and professional growth.

## Careers at UnitedHealthcare
We are continuously looking for talented individuals to join our team. Whether you’re an experienced professional or just starting your career, we offer diverse opportunities in various fields including healthcare, IT, customer service, and management. Our employees are provided with the tools, support, and leadership needed to succeed and make a difference in the lives of our members.

## Join Us
Whether you are seeking health insurance for yourself, your family, or considering a career with us, UnitedHealthcare is here to help navigate your journey towards better health.

### Take the Next Step
- **Shop for a Plan**: [Find a plan that works for you](#).
- **Explore Career Opportunities**: [Join our team today](#).

Together, we can build a healthier future.
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [24]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
stream_brochure("United HealthCare", "https://www.uhc.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.uhc.com/communityplan'}, {'type': 'medicare page', 'url': 'https://www.uhc.com/medicare'}, {'type': 'individuals & families page', 'url': 'https://www.uhc.com/individuals-families'}, {'type': 'employer page', 'url': 'https://www.uhc.com/employer'}, {'type': 'jobs/career page', 'url': 'https://www.uhc.com/employer'}, {'type': 'news articles page', 'url': 'https://www.uhc.com/news-articles'}]}


# UnitedHealthcare Brochure

## Welcome to UnitedHealthcare

At UnitedHealthcare, we understand that health insurance is more than just a policy; it's a commitment to your health and well-being. Our wide range of health insurance plans is designed to meet the diverse needs of individuals, families, businesses, and communities.

---

### Health Plans for Every Stage of Life

Whether you're exploring options for yourself, your family, or your employees, we have the right plan for you:
- **Medicare** - Coverage that extends beyond Original Medicare, offering various Medicare Advantage plans.
- **Medicaid** - Options available for those with low incomes or special circumstances.
- **Individual & Family Plans** - ACA Marketplace plans for individuals and families, available during designated Open Enrollment Periods.
- **Employers** - Comprehensive group health plans designed to support your business and your employees.

Explore our plans and discover the perfect fit for your unique health needs!

---

### Caring Support and Resources

At UnitedHealthcare, we go beyond just providing coverage. Our members receive access to:
- **Personalized Support** - Customized help to choose the best health plan for your situation.
- **UnitedHealthcare App** - Instant access to manage health plan details, request ID cards, and find care options right at your fingertips.
- **Wellness Programs** - Initiatives focused on preventive care to keep you healthy and save on potential medical costs.

---

### Company Culture

Our commitment to caring for our members extends to our employees as well. UnitedHealthcare fosters a culture rooted in:
- **Diversity and Inclusion** - We are dedicated to creating an environment where every individual is valued and respected.
- **Professional Growth** - We offer numerous opportunities for career advancement and skill development.
- **Work-Life Balance** - Recognizing the importance of balance, we provide resources to support both your professional and personal well-being.

---

### Careers at UnitedHealthcare

Join our mission to help people live healthier lives! At UnitedHealthcare, you can expect:
- **A Meaningful Career** - Be part of a team that impacts lives positively every day.
- **Innovative Environment** - Work with forward-thinking professionals committed to health innovation.
- **Comprehensive Benefits** - Enjoy a competitive package that supports well-being and development, including healthcare, retirement savings plans, and more.

---

### Why Choose UnitedHealthcare?

Choosing UnitedHealthcare means choosing a partner dedicated to your health. Our plans provide quality coverage and caring support to empower you at every stage of your life. With innovative solutions and a broad network of providers, we're here to help you navigate your healthcare journey.

Explore our insurance plans today, and let UnitedHealthcare support you in living your healthiest life! 

To learn more about our offerings, visit [UnitedHealthcare.com](https://www.unitedhealthcare.com).

--- 

**Contact Us**  
Have questions or need assistance? Reach out for help in choosing the right plan for you!

**Follow Us**  
Stay connected for updates and health tips on our social media channels!

In [26]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("United HealthCare", "https://www.uhc.com/")

Found links: {'links': [{'type': 'medicare page', 'url': 'https://www.uhc.com/medicare'}, {'type': 'community plan page', 'url': 'https://www.uhc.com/communityplan'}, {'type': 'individuals and families page', 'url': 'https://www.uhc.com/individuals-families'}, {'type': 'employer page', 'url': 'https://www.uhc.com/employer'}, {'type': 'understanding health insurance page', 'url': 'https://www.uhc.com/understanding-health-insurance'}, {'type': 'news articles', 'url': 'https://www.uhc.com/news-articles'}]}


# UnitedHealthcare Company Brochure

---

## About Us
UnitedHealthcare is a leading health insurance provider, dedicated to delivering quality coverage and compassionate support to our members. With a diverse range of health plans tailored for every stage of life, we aim to meet the unique needs of individuals, families, and employers.

---

## Our Health Insurance Plans
### We Offer:
- **Medicare Plans**: Coverage that exceeds Original Medicare, including Medicare Advantage.
- **Medicaid Plans**: Options available for those with low income or special circumstances.
- **Individual & Family Plans**: Explore Affordable Care Act (ACA) Marketplace plans.
- **Employer Plans**: Group coverage options tailored to workplace needs.
- **Supplemental Coverage**: Dental, vision, and other ancillary plans to support overall health needs.

### Open Enrollment
Join us during the Open Enrollment Period (OEP) from **November 1 to January 15**, where you can shop for plans that fit your lifestyle and budget.

---

## Company Culture
At UnitedHealthcare, our culture is rooted in compassion, integrity, and diversity. We believe in fostering an inclusive work environment where all employees feel valued and empowered. We encourage innovation and teamwork, aimed at improving the health and well-being of our members and communities.

---

## Customers & Community
Our diverse customer base includes individuals, families, seniors, and businesses across the U.S. We pride ourselves on a commitment to customer support, ensuring that every member has access to resources that promote preventive care, wellness, and healthcare management.

### Member Support
- **Member Portal**: Access personalized plan benefits online.
- **UnitedHealthcare App**: Manage health plan details, request ID cards, and find care on-the-go.

---

## Careers at UnitedHealthcare
We're always on the lookout for talented individuals who are passionate about making a difference in the healthcare industry. 

### Career Opportunities:
Join a vibrant team that values your contributions and offers:
- **Competitive Salaries**: Fair pay based on industry standards.
- **Comprehensive Benefits**: Health insurance, flexible spending accounts, and retirement plans.
- **Professional Development**: Continuous learning opportunities to grow your skills.

Explore our current job openings and begin your journey with us today!

---

## Get Started
Whether you’re looking for a health plan, considering a career with us, or seeking to partner with us, UnitedHealthcare is here to help. Visit our website to learn more and take the first step towards a healthier future!

[Visit UnitedHealthcare](https://www.unitedhealthcare.com) 

--- 

**UnitedHealthcare – Health Plans for Life's Moments**